In [2]:
import numpy as np

In [3]:
import urllib

In [4]:
import scipy.optimize

In [5]:
import random
from sklearn import datasets, linear_model, metrics

In [6]:
def parseData(fname):
  for l in urllib.urlopen(fname):
    yield eval(l)

In [7]:
print "Reading data..."
data = list(parseData("file:///Users/YW/Dropbox/UCSD/FA2015/CSE255/HW/hw1/beer_50000.json"))
print "done"

Reading data...
done


In [9]:
def feature(datum):
    feat = [1]
    feat.append(datum['beer/ABV'])
    return feat

In [10]:
X = [feature(d) for d in data]

In [11]:
y = [d['review/taste'] for d in data]

In [12]:
theta, residuals, rank, s = np.linalg.lstsq(X, y)

In [13]:
print 'Theta0:%s, Theta1:%s'%(theta[0], theta[1])

Theta0:3.11521115232, Theta1:0.109055074172


In [14]:
X1 = np.matrix(X)
y1 = np.matrix(y)
print np.linalg.inv(X1.T * X1) * X1.T * y1.T

[[ 3.11521115]
 [ 0.10905507]]


In [33]:
def features(datum, degree):
    feat = [1]
    for i in xrange(1, degree+1):
        feat.append(datum['beer/ABV']**i)
    return feat

In [34]:
deg = 0

In [38]:
while True:
    regr = linear_model.LinearRegression(fit_intercept=False)
    X = [features(d, deg) for d in data]
    y = [d['review/taste'] for d in data]
    regr.fit(X, y)
    thisTrainError = np.mean((regr.predict(X)-y) ** 2)
    print "degree:%s\\\\"%(len(X[0])-1)
    print "Train MSE:%s\\\\"% thisTrainError
    print "coefficients:%s\\\\"% regr.coef_
    deg = deg + 1
    if deg > 5:
        break

degree:6\\
Train MSE:0.436387242825\\
coefficients:[  1.17447000e+00   8.02231045e-01  -8.21260380e-02   4.40681142e-03
  -1.39646657e-04   2.35611563e-06  -1.56119050e-08]\\


In [39]:
train = data[:25000]
test = data[25000:]

In [40]:
lastTestingError = 0

In [45]:
deg = 0

In [46]:
while True:
    regr = linear_model.LinearRegression(fit_intercept=False)
    X = [features(d, deg) for d in train]
    y = [d['review/taste'] for d in train]
    X_test = [features(d, deg) for d in test]
    y_test = [d['review/taste'] for d in test]
    regr.fit(X, y)
    thisTestingError = np.mean((regr.predict(X_test) - y_test) ** 2)
    thisTrainError = np.mean((regr.predict(X)-y) ** 2)
    print "Degree:%s\\\\"%(len(X[0])-1)
    print "Train:%s\\\\"% thisTrainError
    print "Test:%s\\\\"% thisTestingError
    print "Coef:%s\\\\"% regr.coef_
    if np.fabs(thisTestingError-lastTestingError) < 0.000001:
        break
    deg = deg + 1
    lastTestingError = thisTestingError

Degree:0\\
Train:0.55835536\\
Test:0.46912512\\
Coef:[ 3.9092]\\
Degree:1\\
Train:0.483983105115\\
Test:0.423776528023\\
Coef:[ 2.99503282  0.11690802]\\
Degree:2\\
Train:0.471743067557\\
Test:0.427256126\\
Coef:[ 2.62007309  0.20716481 -0.00496806]\\
Degree:3\\
Train:0.457832195847\\
Test:0.432820707084\\
Coef:[  1.57847740e+00   5.19869113e-01  -2.97470415e-02   3.97626061e-04]\\
Degree:4\\
Train:0.451641221709\\
Test:0.438301732573\\
Coef:[  7.17629022e-01   8.26164532e-01  -6.32189636e-02   1.67076516e-03
  -1.40075070e-05]\\
Degree:5\\
Train:0.451335575196\\
Test:0.439820774965\\
Coef:[  1.16389773e+00   6.12905604e-01  -2.85943943e-02  -6.28209940e-04
   4.41325901e-05  -4.79191916e-07]\\
Degree:6\\
Train:0.450769302041\\
Test:0.443012263638\\
Coef:[  1.94901771e+00   1.79935890e-01   5.75772252e-02  -8.45991886e-03
   3.82283840e-04  -7.06992367e-06   4.63100297e-08]\\
Degree:7\\
Train:0.45074188066\\
Test:0.442575342636\\
Coef:[  1.70847546e+00   3.47096902e-01   1.39022021e-02

In [108]:
print X[0]
print X[1]
print regr.coef_

[1, 5.0, 25.0, 125.0, 625.0, 3125.0, 15625.0, 78125.0, 390625.0, 1953125.0, 9765625.0, 48828125.0, 244140625.0, 1220703125.0, 6103515625.0, 30517578125.0, 152587890625.0, 762939453125.0, 3814697265625.0, 19073486328125.0, 95367431640625.0, 476837158203125.0, 2384185791015625.0]
[1, 6.2, 38.440000000000005, 238.32800000000003, 1477.6336000000001, 9161.32832, 56800.23558400001, 352161.4606208001, 2183401.0558489603, 13537086.546263555, 83929936.58683404, 520365606.8383711, 3226266762.397901, 20002853926.866985, 124017694346.57532, 768909704948.767, 4767240170682.355, 29556889058230.605, 183252712161029.75, 1136166815398384.5, 7044234255469984.0, 4.36742523839139e+16, 2.707803647802662e+17]
[  1.31263710e-53  -5.38892493e-32  -3.23662641e-36   3.15709069e-39
   2.62616297e-42   4.17109278e-46   1.02960295e-44   3.07504109e-43
   9.10714282e-42   2.66863746e-40   7.71445794e-39   2.19155799e-37
   6.08652461e-36   1.64066299e-34   4.24821298e-33   1.04034086e-31
   2.35023179e-30   4.68797

In [138]:
print lastTestingError
print thisTestingError

15.8358415497
15.8358416122
